# Advent of Code 2023: Day 20
https://adventofcode.com/2023/day/20


## Part 1
Count the number of low and high pulses sent after 1000 button presses

In [1]:
from math import lcm

### Get the data into a list of strings

In [2]:
myfile = open('input.txt', 'r')
data = myfile.read()
data_list = data.split('\n')

### Create function to process the list into a dictionary
The items in the dictionary will take different forms depending on their type of module.

A broadcast module:
* name: [list of outputs]
  * 'broadcaster': ['a', 'b', 'c']

A flip-flop module:
* name: ['off or on', [list of outputs]]
  * 'a': ['off', ['b']]

A conjunction module:
* name: [[dict of inputs and their last pulse], [list of outputs]]
  * 'e': [{'c': 'low', 'd': 'high'}, ['f', 'g']]

Also create an identical copy of the dictionary for later comparison (turned out to not be needed)

In [3]:
def preprocess(data_list):
  data_dict = {}
  data_dict_start = {}
  for line in data_list:
    tmp = line.split(' -> ')
    if tmp[0] == 'broadcaster':
      data_dict[tmp[0]] = tmp[1].split(', ')
      data_dict_start[tmp[0]] = tmp[1].split(', ')
    elif tmp[0][0] == '%':
      data_dict[tmp[0][1:]] = ['off', tmp[1].split(', ')]
      data_dict_start[tmp[0][1:]] = ['off', tmp[1].split(', ')]
    elif tmp[0][0] == '&':
      data_dict[tmp[0][1:]] = [{}, tmp[1].split(', ')]
      data_dict_start[tmp[0][1:]] = [{}, tmp[1].split(', ')]

  for key, value in data_dict.items():
    if key != 'broadcaster':
      for module in value [1]:
        if module in data_dict.keys():
          if isinstance(data_dict[module][0], dict):
            data_dict[module][0][key] = 'low'
            data_dict_start[module][0][key] = 'low'
  return data_dict, data_dict_start

### Create function which handles a module recieving a pulse

Does not need to be implemented for the broadcaster module

In [4]:
def recievePulse(strength, sender, reciever, data_dict):
  tmp = data_dict[reciever][0]
  if isinstance(tmp, str):
    if strength == 'high':
      return None, data_dict
    else:
      if tmp == 'off':
        data_dict[reciever][0] = 'on'
        return 'high', data_dict
      else:
        data_dict[reciever][0] = 'off'
        return 'low', data_dict
  else:
    data_dict[reciever][0][sender] = strength
    if all(val == 'high' for val in data_dict[reciever][0].values()):
      return 'low', data_dict
    else:
      return 'high',data_dict

### Create function to simulate the push of a button

Create a queue of pulses that have to be sent and add the initial pulses sent out by the broadcaster module to it. Afterwards, process pulses until there are no more to send.

In [5]:
def pushButton(pulse_count, data_dict, target = False):
  send_q = []
  pulse_count['low'] +=1
  target_count = {
      'low': 0,
      'high': 0
  }
  for module in data_dict['broadcaster']:
    send_q.append(['low', 'broadcaster', module])
  while send_q:
    cur = send_q.pop(0)
    pulse_count[cur[0]] +=1
    strength, data_dict = recievePulse(cur[0], cur[1], cur[2], data_dict)
    if strength is not None:
      for module in data_dict[cur[2]][1]:
        if module in data_dict.keys():
          send_q.append([strength, cur[2], module])
          if cur[2] == target:
            target_count[strength] += 1
        else:
          pulse_count[strength] +=1
  if target:
    return pulse_count, data_dict, target_count
  else:
    return pulse_count, data_dict

### Create function to push the button **_n_** number of times

To speed it up, look for a cycle, i.e. when the state of all modules is back to their initial settings. Then the pulse counts can be multiplied with the number of cycles, and if this does not add up evenly to n, push the button a few more times.

Again, this did not end up being needed as _n_ is lower than the length of a cycle.

In [6]:
def pushButtons(pulse_count, data_dict, data_dict_start, n):
  cycle_length = 0
  for i in range(n):
    pulse_count, data_dict = pushButton(pulse_count, data_dict)
    if data_dict == data_dict_start:
      cycle_length = i+1
      break

  if cycle_length != 0:
    remain = n % cycle_length
    mult = n // cycle_length
    pulse_count['low'] *= mult
    pulse_count['high'] *= mult
    for i in range(remain):
      pulse_count, data_dict = pushButton(pulse_count, data_dict)
  return pulse_count, data_dict

### Count the number of low and high pulses after pressing the button 1000 times and multiply the two

In [7]:
pulse_count = {
      'low': 0,
      'high': 0
  }
n = 1000
data_dict, data_dict_start = preprocess(data_list)
pulse_count, data_dict = pushButtons(pulse_count, data_dict, data_dict_start, n)
pulse_count['low'] * pulse_count['high']

711650489

## Part 2
Count the number of button presses required to deliver a single low pulse to the module named rx

### Create function to find how many button presses it takes to deliver a single low pulse to the target node

To speed up this process, go backwards and look where the target should get the low pulse from. As this is a conjunction module, retrieve a list of its input modules.

Then, for each of these modules, find how many times the button should be pressed until the module sends a single high pulse (if each of the modules sends a single high pulse, the conjunction module should send a single low pulse to the target) and store it in a list.

Finally, go through the list and perform **lcm** to find where these numbers will converge.

In [8]:
def startMachine(pulse_count, data_dict, target):
  i = 0
  j = 0

  for key, value in data_dict.items():
    if value[1] == [target]:
      targets = list(value[0].keys())
      break

  pushes = []
  while j < len(targets):
    i+=1
    pulse_count, data_dict, target_count = pushButton(pulse_count, data_dict, targets[j])
    if target_count['high'] == 1:
      pushes.append(i)
      i = 0
      j +=1
      data_dict,_ = preprocess(data_list)

  total = 1
  for num in pushes:
    total = lcm(total, num)
  return total

### Find the number of pushes required

In [9]:
data_dict,_ = preprocess(data_list)
startMachine(pulse_count, data_dict, 'rx')

219388737656593